In [1]:
#!pip install unzip
#!pip install wget

In [2]:
# download files
#!mkdir data
#!wget "https://www.dropbox.com/s/948jr9cfs7fgj99/UBER.zip?dl=1" -O data/UBER.zip
#!unzip data/UBER.zip -d data
#!pip3 install pytesseract

In [3]:
# set text wrapping
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [4]:
from llama_index import download_loader, GPTVectorStoreIndex, ServiceContext
from pathlib import Path

Ingest Unstructured Data Through the Unstructured.io Reader
Leverage the capabilities of Unstructured.io HTML parsing. Downloaded through LlamaHub.

In [5]:
years = [2022, 2021, 2020, 2019]

In [6]:
UnstructuredReader = download_loader("UnstructuredReader", refresh_cache=True)

In [7]:
loader = UnstructuredReader()
doc_set = {}
all_docs = []
for year in years:
    year_docs = loader.load_data(file=Path(f'./data/UBER/UBER_{year}.html'), split_documents=False)
    # insert year metadata into each year
    for d in year_docs:
        d.extra_info = {"year": year}
    doc_set[year] = year_docs
    all_docs.extend(year_docs)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\irfan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\irfan\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...
INFO:unstructured:Reading document from string ...
INFO:unstructured:Reading document ...


## Setup a Vector Index for each SEC filing

We setup a separate vector index for each SEC filing from 2019-2022.

We also optionally initialize a "global" index by dumping all files into the vector store.

In [8]:
import os
os.environ["OPENAI_API_KEY"] = 'Enter-API-KEY'


In [9]:
# initialize simple vector indices + global vector index
# NOTE: don't run this cell if the indices are already loaded! 
index_set = {}
service_context = ServiceContext.from_defaults(chunk_size_limit=512)
for year in years:
    cur_index = GPTVectorStoreIndex.from_documents(doc_set[year], service_context=service_context)
    index_set[year] = cur_index
    

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 232797 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 241424 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 257154 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 246480 tokens


In [10]:
# Load indices from disk
index_set = {}
for year in years:
    index_set[year] = cur_index

# Composing a Graph to synthesize answers across 10-K filings (2019-2022)

We want our queries to aggregate/synthesize information across all 10-K filings. To do this, we define a List index on top of the 4 vector indices.

In [11]:
from llama_index import GPTListIndex, LLMPredictor
from langchain import OpenAI
from llama_index.indices.composability import ComposableGraph

In [12]:
# set summary text for each doc
index_summaries = [f"UBER 10-k Filing for {year} fiscal year" for year in years]

In [13]:
# set number of output tokens
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, max_tokens=512))
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)

In [14]:
# define a list index over the vector indices
# allows us to synthesize information across each index
graph = ComposableGraph.from_indices(
    GPTListIndex, 
    [index_set[y] for y in years], 
    index_summaries=index_summaries,
    service_context=service_context
)

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 0 tokens


## Setting up the Chatbot Agent

We use Langchain to define the outer chatbot abstraction. We use LlamaIndex as a core Tool within this abstraction.

In [15]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

from llama_index.langchain_helpers.agents import LlamaToolkit, create_llama_chat_agent, IndexToolConfig

In [16]:
# define a decompose transform
from llama_index.indices.query.query_transform.base import DecomposeQueryTransform
from llama_index.query_engine.transform_query_engine import TransformQueryEngine
decompose_transform = DecomposeQueryTransform(
    llm_predictor, verbose=True
)

# define custom query engines
custom_query_engines = {}
for index in index_set.values():
    query_engine = index.as_query_engine()
    query_engine = TransformQueryEngine(
        query_engine,
        query_transform=decompose_transform,
        transform_extra_info={'index_summary': index.index_struct.summary},
    )
    custom_query_engines[index.index_id] = query_engine
custom_query_engines[graph.root_id] = graph.root_index.as_query_engine(
    response_mode='tree_summarize',
    verbose=True,
)

# construct query engine
graph_query_engine = graph.as_query_engine(custom_query_engines=custom_query_engines)

In [17]:
# index configs
index_configs = []
for y in range(2019, 2023):
    query_engine = index_set[y].as_query_engine(
        similarity_top_k=3,
    )
    tool_config = IndexToolConfig(
        query_engine=query_engine, 
        name=f"Vector Index {y}",
        description=f"useful for when you want to answer queries about the {y} SEC 10-K for Uber",
        tool_kwargs={"return_direct": True, "return_sources": True},
    )
    index_configs.append(tool_config)
    
# graph config
graph_config = IndexToolConfig(
    query_engine=graph_query_engine,
    name=f"Graph Index",
    description="useful for when you want to answer queries that require analyzing multiple SEC 10-K documents for Uber.",
    tool_kwargs={"return_direct": True, "return_sources": True},
    return_sources=True
)

toolkit = LlamaToolkit(
    index_configs=index_configs,
    graph_configs=[graph_config]
)

In [18]:
memory = ConversationBufferMemory(memory_key="chat_history")
llm=OpenAI(temperature=0)
agent_chain = create_llama_chat_agent(
    toolkit,
    llm,
    memory=memory,
    verbose=True
)

In [19]:
agent_chain.run(input="hi, i am Irfan")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: Hi Irfan, nice to meet you! How can I help you today?

> Finished chain.


'Hi Irfan, nice to meet you! How can I help you today?'

In [20]:
agent_chain.run(input="What were some of the biggest risk factors in 2020 for Uber?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Vector Index 2020
Action Input: Risk Factors

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 3 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1371 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens



Observation: {'answer': '\nRisks related to our business include: competition from existing, well-established, and low-cost alternatives; low barriers to entry; low switching costs; and well-capitalized competitors in nearly every major geographic region. We also face risks related to data security and privacy, including potential security breaches, data loss, and undesirable activities on our platform. Additionally, our platform is highly technical, and any undetected errors could adversely affect our business.', 'sources': [{'start': 219104, 'end': 221323, 'ref_doc_id': 'be675e57-312c-40fb-b8ec-dcb4b749a67c', 'score': 0.8055922198196064}, {'start': 70976, 'end': 73149, 'ref_doc_id': 'be675e57-312c-40fb-b8ec-dcb4b749a67c', 'score': 0.8007849581582945}, {'start': 259320, 'end': 261573, 'ref_doc_id': 'be675e57-312c-40fb-b8ec-dcb4b749a67c', 'score': 0.7940940753189837}]}

> Finished chain.


"{'answer': '\\nRisks related to our business include: competition from existing, well-established, and low-cost alternatives; low barriers to entry; low switching costs; and well-capitalized competitors in nearly every major geographic region. We also face risks related to data security and privacy, including potential security breaches, data loss, and undesirable activities on our platform. Additionally, our platform is highly technical, and any undetected errors could adversely affect our business.', 'sources': [{'start': 219104, 'end': 221323, 'ref_doc_id': 'be675e57-312c-40fb-b8ec-dcb4b749a67c', 'score': 0.8055922198196064}, {'start': 70976, 'end': 73149, 'ref_doc_id': 'be675e57-312c-40fb-b8ec-dcb4b749a67c', 'score': 0.8007849581582945}, {'start': 259320, 'end': 261573, 'ref_doc_id': 'be675e57-312c-40fb-b8ec-dcb4b749a67c', 'score': 0.7940940753189837}]}"

In [21]:
cross_query_str = (
    "Compare/contrast the risk factors described in the Uber 10-K across years. Give answer in bullet points."
)

response = agent_chain.run(input=cross_query_str)



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: Vector Index 2019, Vector Index 2020, Vector Index 2021, Vector Index 2022
Action Input: Risk factors
Observation: Vector Index 2019, Vector Index 2020, Vector Index 2021, Vector Index 2022 is not a valid tool, try another one.
Thought: Do I need to use a tool? No
AI: The risk factors described in the Uber 10-K have changed over the years. In 2019, the risk factors included competition from existing, well-established, and low-cost alternatives; low barriers to entry; low switching costs; and well-capitalized competitors in nearly every major geographic region. In 2020, the risk factors included the potential for data security and privacy breaches, data loss, and undesirable activities on the platform. In 2021, the risk factors included the potential for regulatory changes, the impact of the COVID-19 pandemic, and the potential for increased competition. In 2022, the risk factors included the potentia

In [22]:
# reinitialize agent
memory = ConversationBufferMemory(memory_key="chat_history")
llm=OpenAI(temperature=0)
agent_chain = create_llama_chat_agent(
    toolkit,
    llm,
    memory=memory,
)

In [ ]:
while True:
    text_input = input("User: ")
    response = agent_chain.run(input=text_input)
    print(f'Agent: {response}')

User: How are You?
Agent: Hi there! I'm doing great, thanks for asking. How can I help you?
User: Compare/contrast the risk factors described in the Uber 10-K across years. Give answer in bullet points.
Agent: Comparing and contrasting the risk factors described in the Uber 10-K across years can be done by looking at the risk factors section of each 10-K. Generally, the risk factors section will discuss the potential risks associated with the company's operations, such as economic conditions, competitive pressures, legal and regulatory risks, and other factors. By comparing the risk factors across years, you can get a better understanding of how the company's risk profile has changed over time. Some of the key risk factors to look for include: 

• Economic conditions: How has the economic environment changed over time and how has this impacted the company's operations?
• Competitive pressures: How has the competitive landscape changed over time and how has this impacted the company's o